# 一、Pytorch基本操作考察

#### 1.1 使用Tensor初始化一个 1x3 的矩阵 M 和一个 2x1 的矩阵 N，对两矩阵进行减法操作（要求实现三种不同的形式），给出结果并分析三种方式的不同（如果出现报错，分析报错的原因），同时需要指出在计算过程中发生了什么.

In [63]:
# Step 1: 导入Pytorch
import torch

# 检查设备可用性
if torch.cuda.is_available():
    device = torch.device("cuda")  # 如果支持cuda，则选择cuda
elif torch.backends.mps.is_available():
    device = torch.device("mps")  # 如果支持Apple M系芯片，则选择mps
else:
    device = torch.device("cpu")  # 否则选择cpu
print("当前设备：", device)

当前设备： mps


In [64]:
# Step 2: 初始化矩阵M和N
M = torch.randn(1,3)
N = torch.randn(2,1)
print(M)
print(N)

tensor([[ 0.6477,  0.7037, -1.1246]])
tensor([[ 0.3003],
        [-0.1905]])


为了更好地分析结果，我们给M、N张量赋予特值

In [65]:
M = torch.tensor([[1,2,3]])
N = torch.tensor([[1],[2]])

# 将张量发送到所选设备
M = M.to(device)
N = N.to(device)

print(M)
print(N)

tensor([[1, 2, 3]], device='mps:0')
tensor([[1],
        [2]], device='mps:0')


In [66]:
# Step 3: 对两矩阵进行减法操作 （三种实现形式）
# 减法形式一：
print(M-N) 
# 减法形式二：
print(M.sub(N))
# 减法形式三：
print(torch.sub(M,N))  # print(torch.subtract(M,N)) # 同上

tensor([[ 0,  1,  2],
        [-1,  0,  1]], device='mps:0')
tensor([[ 0,  1,  2],
        [-1,  0,  1]], device='mps:0')
tensor([[ 0,  1,  2],
        [-1,  0,  1]], device='mps:0')


In [67]:
# 减法形式四：
print(M.sub_(N))

RuntimeError: output with shape [1, 3] doesn't match the broadcast shape [2, 3]

In [68]:
Mx = torch.tensor([1,2,3])
Nx = torch.tensor([4,5,6])
print(Mx,Nx)
print(Mx.sub(Nx),'\t',Mx,'\t',Nx)
print(Mx.sub_(Nx),'\t',Mx,'\t',Nx)

tensor([1, 2, 3]) tensor([4, 5, 6])
tensor([-3, -3, -3]) 	 tensor([1, 2, 3]) 	 tensor([4, 5, 6])
tensor([-3, -3, -3]) 	 tensor([-3, -3, -3]) 	 tensor([4, 5, 6])


由上我们可见：使用张量的sub_方法，提示报错，原因是M、N的形状不同，无法广播，无法进行减法操作。
重新定义相同大小的Mx和Nx张量，并使用sub_方法进行减法操作。
可以发现，sub_是在在原张量上进行减法操作，不会产生新的张量，而是直接修改原张量的值；而sub方法则会返回一个新的张量，不会修改原张量的值。

&ensp; 由Step3的结果可以发现，三种方法的输出结果都是相同的，都是对应元素相减得到的矩阵。在计算过程中，Pytorch会自动进行广播操作，将1x3的矩阵M和2x1的矩阵N进行广播，使得它们的形状相同，从而进行元素级别的减法操作。

其计算过程为：
&ensp; [1,2,3]会被广播为[[1,2,3],[1,2,3]]
&ensp; [[1],[2]]会被广播为[[1,1,1],[2,2,2]]
&ensp; [[1,2,3],[1,2,3]] - [[1,1,1],[2,2,2]] = [[0,1,2],[-1,0,1]]
&ensp; 即得到了以上的结果。

#### 1.2 ① 利用 Tensor 创建两个大小分别 3x2 和 4x2 的随机数矩阵 P 和 Q，要求服从均值为 0，标准差0.01为的正态分布；② 对第二步得到的矩阵 Q 进行形状变换得到 Q 的转置 Q<sup>T</sup> ；③ 对上述得到的矩阵 P 和矩阵 Q<sup>T</sup>求矩阵相乘

In [72]:
# Step1: N(0,0.01)创建
# 法一：
P = torch.randn(3,2)*0.01
Q = torch.randn(4,2)*0.01
# 法二：
import numpy as np
P = torch.tensor(np.random.normal(0,0.01,size=(3,2)))
Q = torch.tensor(np.random.normal(0,0.01,size=(4,2)))
print(P)
print(Q)

tensor([[ 0.0032, -0.0106],
        [-0.0138,  0.0139],
        [-0.0042,  0.0075]], dtype=torch.float64)
tensor([[-3.4224e-05, -1.3588e-02],
        [-1.5247e-02, -5.9312e-03],
        [ 7.3446e-03,  3.1417e-03],
        [-1.5942e-02, -1.8460e-03]], dtype=torch.float64)


In [79]:
# Step 2: 对Q进行转置
Qt = Q.t()
print(Qt)

tensor([[-3.4224e-05, -1.5247e-02,  7.3446e-03, -1.5942e-02],
        [-1.3588e-02, -5.9312e-03,  3.1417e-03, -1.8460e-03]],
       dtype=torch.float64)


In [82]:
print(torch.mm(P,Qt))

tensor([[ 1.4332e-04,  1.3260e-05, -9.3918e-06, -3.2112e-05],
        [-1.8853e-04,  1.2864e-04, -5.8008e-05,  1.9509e-04],
        [-1.0175e-04,  1.8909e-05, -6.9747e-06,  5.2432e-05]],
       dtype=torch.float64)


#### 1.3 给定公式 y3 = y1 + y2 = x<sup>2</sup> + x<sup>3</sup>，且 x=1。利用学习所得到的Tensor的相关知识，求y3对x的梯度，即dy3/dx。 <br>&ensp;&ensp;要求在计算过程中，在计算x<sup>3</sup>时中断梯度的追踪，观察结果并进行原因分析.<br>提示：可使用 with torch.no_grad(),举例：<br> with torch.no_grad():<br>&ensp;&ensp;y=x*5

In [85]:
import torch
x = torch.tensor(1.0, requires_grad=True)

# 计算 y1
y1 = x ** 2

# 中断梯度追踪
with torch.no_grad():
    # 计算 y2
    y2 = x ** 3

# 计算 y3
y3 = y1 + y2

# 计算梯度dy3/dx，反向传播
y3.backward()

#输出x的梯度值
print(x.grad)

tensor(2.)


对如上结果进行分析：
如果不中断y2=x^3的梯度传递，则最终 y3对x 的梯度值应为5，而实际为 2，则说明 y2 在计算过程中没有被追踪梯度，因此 y3 的梯度只与 y1 相关。

# 二、动手实现 logistic 回归

## 2.1 从 0 实现

In [40]:
import torch
import numpy as np


class Model:
    # 定义模型
    def logistic(self, X, w, b):
        y = self.sigmoid(torch.matmul(X, w) + b)  # 通过sigmoid函数将线性回归的输出转换为概率
        return y

    # 定义激活函数
    def sigmoid(self, x):
        return 1 / (1 + torch.exp(-x))

    # 定义损失函数（二元交叉熵损失/对数损失函数）
    def loss(self, y_pred, y_true):
        return -torch.mean(y_true * torch.log(y_pred) + (1 - y_true) * torch.log(1 - y_pred))

    # 定义优化算法 SGD
    def sgd(self, w, b, lr):
        with torch.no_grad():
            w -= lr * w.grad
            b -= lr * b.grad


class Train:
    def __init__(self, model, features, labels, lr, epochs):
        self.model = model
        self.features = features
        self.labels = labels
        self.lr = lr
        self.epochs = epochs
        self.w = torch.normal(0, 0.01, size=(features.shape[1], 1), requires_grad=True)  # 权重
        self.b = torch.zeros(1, requires_grad=True)  # 偏置

        # 读取数据集

    def data_iter(self, batch_size, features, labels):
        num_examples = len(features)
        indices = list(range(num_examples))
        np.random.shuffle(indices)
        for i in range(0, num_examples, batch_size):
            batch_indices = torch.tensor(indices[i: min(i + batch_size, num_examples)])
            yield features[batch_indices], labels[batch_indices]

    # 训练
    def train(self):
        for epoch in range(self.epochs):
            for X, y in self.data_iter(batch_size=10, features=self.features, labels=self.labels):
                y_pred = self.model.logistic(X, self.w, self.b)
                l = Model.loss(model ,y_pred, y)

                # l.backward()
                l.sum().backward()
                Model.sgd(model ,self.w, self.b, self.lr)
            with torch.no_grad():
                y_pred = self.model.logistic(features, self.w, self.b)
                l = Model.loss(model ,y_pred, labels)
                print(f'epoch {epoch + 1}, loss {float(l.mean()):f}')

    def predict(self, X):
        y_pred = self.model.logistic(X, self.w, self.b)  # 预测值
        return y_pred

    # 保存模型
    def save_model(self, path):
        torch.save({
            'w': self.w,
            'b': self.b
        }, path)


class DataProducer:
    # 生成正态线性数据集
    def synthetic_data(self, w, b, num_samples):
        X = torch.normal(0, 1, (num_samples, len(w)))
        y = torch.matmul(X, w) + b
        y += torch.normal(0, 0.01, y.shape)
        return X, y.reshape((-1, 1))

    # 生成二分类数据集
    def synthesize_data_binarization(self, num_examples):
        n_data = torch.ones(num_examples, 2)  # 数据的基本形态 num_examples x 2

        x1 = torch.normal(2 * n_data, 1)  # shape=(50, 2)
        y1 = torch.zeros(num_examples)  # 类型0 shape=(50, 1)

        x2 = torch.normal(-2 * n_data, 1)  # shape=(50, 2)
        y2 = torch.ones(num_examples)  # 类型1 shape=(50, 1)

        # 注意 x, y 数据的数据形式一定要像下面一样 (torch.cat 是合并数据)
        x = torch.cat((x1, x2), 0).type(torch.FloatTensor)
        y = torch.cat((y1, y2), 0).type(torch.FloatTensor)
        return x, y


if __name__ == '__main__':
    # 准备数据集
    data_producer = DataProducer()
    data = data_producer.synthesize_data_binarization(num_examples=3000)  # 生成1000个样本
    features, labels = data  # 分别获取特征和标签

    # 初始化模型参数
    model = Model()

    # 训练
    batch_size = 100  # 每批数据集大小
    train = Train(model, features, labels, lr=0.003, epochs=3)
    train.train()


epoch 1, loss 0.710352
epoch 2, loss 0.712597
epoch 3, loss 0.695746


## 2.2 利用torch.nn实现

In [87]:
import torch
import torch.nn as nn





None


# 三、动手实现 softmax 回归

## 3.1 从0实现

In [ ]:
import torch
import numpy as np
class softmax():
    





## 3.2 利用torch.nn实现

In [ ]:
import torch
import torch.nn as nn



